Notebook responsável por criar a tabela de vendors na camada gold

In [0]:
import sys
import os
sys.path.append('../utils') 

from utils import create_update_table_metadata

In [0]:
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
folder_path = os.path.dirname(notebook_path)

silver_table_name = f"ifood_case.silver.tb_dim_vendor"
table_name = "tb_vendor"
gold_table_name = f"ifood_case.gold.{table_name}"

In [0]:
# método responsável por incluir os metadados nas colunas da tabela. 
# ele lê um arquivo yaml com a configuração das colunas e atualiza os metadados
def create_update_column_metadata(table_name, file_path):
    try:
        with open(file_path, "r") as file:
            column_comments = yaml.safe_load(file)["columns"]

            for column, comment in column_comments.items():
                spark.sql(f"ALTER TABLE {table_name} ALTER COLUMN {column} COMMENT '{comment}'")
    except Exception as e:
        print(f"Error during update column metadata: {e}")

In [0]:
df = spark.read.table(silver_table_name)

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable(f"{gold_table_name}")

In [0]:
metadata_file_path = f"/Workspace/{folder_path}/metadata/{table_name}.yaml"
create_update_table_metadata(spark, silver_table_name, metadata_file_path)